In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [5]:
import pandas as pd
import numpy as np
import torch

import src.preprocessing as pre
import plotly.graph_objects as go
import src.utils as utils
from torch.utils.data import DataLoader

In [ ]:
train, test, mappings, meta = pre.separate_year('../data/meta.json','../data/y.csv','../data/mappings.json')

FileNotFoundError: [Errno 2] No such file or directory: '../data/y.csv'

In [ ]:
model = utils.create_model(train,num_epochs=5)

Epoch [1/200], Loss: 4392918.8643
Epoch [2/200], Loss: 3869970.7727
Epoch [3/200], Loss: 3144702.2990
Epoch [4/200], Loss: 2898099.0163
Epoch [5/200], Loss: 2704800.5703


In [38]:
preds = utils.predict_gridded_harvest(model,test)

In [6]:
actuals = test.Y.detach().numpy()

In [7]:
preds[:,0,:]

array([[-1.67963490e-01, -4.80185971e-02, -9.45493281e-02, ...,
         1.47769604e+03,  1.28266064e+03,  9.57380005e+02],
       [-2.86049932e-01, -4.67371717e-02, -1.20143406e-02, ...,
         1.27532703e+03,  1.10685413e+03,  8.26156921e+02],
       [-9.63351578e-02, -8.83865282e-02, -8.43334049e-02, ...,
         1.63770862e+03,  1.42156812e+03,  1.06085840e+03],
       ...,
       [-1.16921134e-01, -1.00091070e-01, -6.37571663e-02, ...,
         1.41899927e+03,  1.23196594e+03,  9.19646301e+02],
       [-1.03645578e-01, -1.04080096e-01, -5.66823296e-02, ...,
         1.37729041e+03,  1.19573279e+03,  8.92640076e+02],
       [-5.31520136e-02, -1.37318864e-01, -5.68268187e-02, ...,
         1.60494751e+03,  1.39340149e+03,  1.03991821e+03]])

In [23]:
production_plan = pre.decode(meta,mappings).reset_index(drop=True)

In [25]:
current_week = 14
harvest_length = 20
initial_buffer = 5
indices_dict = {'Class':{},'Type':{},'Ranch':{},'Variety':{}}
# Populate indices_dict with the indices of production_plan for Class, Ranch, Type
for column in ['Class', 'Ranch', 'Type', 'Variety']:
    if column in production_plan.columns:
        unique_values = production_plan[column].unique()
        indices_dict[column] = {value: production_plan.index[production_plan[column] == value].tolist() for value in unique_values}

In [26]:
def get_indices(column,vals,indices_dict):
    return np.logical_or.reduce([indices_dict[column][val] for val in vals])

In [27]:
def season_shift(production_plan,preds):
    # Convert preds from weeks after transplant to week of the year
    week_transplanted = production_plan['WeekTransplanted'].values
    num_weeks = preds.shape[1]
    
    # Create a new array to hold the shifted predictions
    shifted_preds = np.zeros_like(preds)
    
    for i in range(preds.shape[0]):
        # Calculate the starting week for each row
        start_week = week_transplanted[i]
        
        # Shift the predictions to the right by the start_week
        if start_week < num_weeks:
            shifted_preds[i, start_week:] = preds[i, :num_weeks - start_week]
    
    return shifted_preds